## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-19-06-39-12 +0000,bbc,Entire church begins two-day journey across Sw...,https://www.bbc.com/news/articles/cde3xp4xlw9o...
1,2025-08-19-06-32-16 +0000,nypost,RI prosecutor threatens cops to shut off body...,https://nypost.com/2025/08/19/us-news/ri-prose...
2,2025-08-19-06-31-42 +0000,nyt,Air Canada Flight Attendants Say They Will Def...,https://www.nytimes.com/2025/08/17/world/canad...
3,2025-08-19-06-30-07 +0000,nyt,Talks Restart as Air Canada Flight Attendants’...,https://www.nytimes.com/2025/08/18/world/canad...
4,2025-08-19-06-24-46 +0000,bbc,Soho House bought for £2bn as Ashton Kutcher j...,https://www.bbc.com/news/articles/ckgjy4d01jwo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2336/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
82,trump,51
77,zelensky,27
76,ukraine,22
35,house,18
28,will,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
226,2025-08-18-11-37-54 +0000,bbc,"'No going into Nato by Ukraine,' says Trump as...",https://www.bbc.com/news/articles/cm21j1ve817o...,153
152,2025-08-18-18-02-57 +0000,nypost,Trump ‘very happy’ after Ukraine peace summit ...,https://nypost.com/2025/08/18/us-news/trump-ze...,146
235,2025-08-18-10-27-12 +0000,nypost,Harrowing photo shows teen killed in Russian s...,https://nypost.com/2025/08/18/world-news/russi...,144
25,2025-08-19-02-43-00 +0000,wsj,Trump Pushes for Putin and Zelensky Meeting in...,https://www.wsj.com/world/trump-pushes-for-pea...,139
224,2025-08-18-12-03-49 +0000,wapo,"What to expect as Zelensky, European leaders m...",https://www.washingtonpost.com/world/2025/08/1...,129


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
226,153,2025-08-18-11-37-54 +0000,bbc,"'No going into Nato by Ukraine,' says Trump as...",https://www.bbc.com/news/articles/cm21j1ve817o...
125,66,2025-08-18-19-54-53 +0000,nypost,Putin claims Ukraine should give up Donetsk be...,https://nypost.com/2025/08/18/world-news/putin...
206,51,2025-08-18-13-53-28 +0000,cbc,Air Canada flight attendants' union says it wi...,https://www.cbc.ca/news/politics/air-canada-st...
55,43,2025-08-19-00-22-32 +0000,nypost,Furious Harvard researcher claims he was pumme...,https://nypost.com/2025/08/18/us-news/furious-...
52,41,2025-08-19-00-31-37 +0000,startribune,Are you State Fair ready? Here’s what you need...,https://www.startribune.com/mn-state-fair/6014...
77,36,2025-08-18-23-13-00 +0000,wsj,Texas Democrats ended a two-week walkout over ...,https://www.wsj.com/politics/policy/texas-demo...
235,35,2025-08-18-10-27-12 +0000,nypost,Harrowing photo shows teen killed in Russian s...,https://nypost.com/2025/08/18/world-news/russi...
90,33,2025-08-18-22-20-19 +0000,nypost,MSNBC gets mercilessly mocked over new MS NOW ...,https://nypost.com/2025/08/18/media/msnbc-gets...
143,32,2025-08-18-18-32-30 +0000,latimes,Newsmax to pay $67 million to settle Dominion ...,https://www.latimes.com/entertainment-arts/bus...
27,28,2025-08-19-02-27-43 +0000,nypost,Elon Musk baby mama Ashley St. Clair reveals n...,https://nypost.com/2025/08/18/us-news/elon-mus...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
